# PSNC Connector demo
based on https://github.com/eclipse-edc/Samples

In [7]:
import requests
import json

### Demo setup

In [9]:
# for paas:

PROVIDER_URL = "https://provider-edc-connector.apps.paas-dev.psnc.pl"
CONSUMER_URL = "https://consumer-edc-connector.apps.paas-dev.psnc.pl"

default_headers = {
    "Content-Type": "application/json"
    ,"X-API-KEY": "fifi-info-zaneta"
}

## for local:
LOCALHOST = "http://localhost"
CONSUMER_CONTAINER = "http://consumer-connector"
PROVIDER_CONTAINER = "http://provider-connector"
# CONSUMER_URL = LOCALHOST
# PROVIDER_URL = LOCALHOST

In [10]:
PROVIDER_API = f"{PROVIDER_URL}/api"
PROVIDER_CONTROL = f"{PROVIDER_URL}/control"
PROVIDER_MANAGEMENT = f"{PROVIDER_URL}/management"
PROVIDER_PROTOCOL = f"{PROVIDER_URL}/protocol"
PROVIDER_PUBLIC = f"{PROVIDER_URL}/public"

CONSUMER_API = f"{CONSUMER_URL}/api"
CONSUMER_CONTROL = f"{CONSUMER_URL}/control"
CONSUMER_MANAGEMENT = f"{CONSUMER_URL}/management"
CONSUMER_PROTOCOL = f"{CONSUMER_URL}/protocol"
CONSUMER_PUBLIC = f"{CONSUMER_URL}/public"

# PROVIDER_API = f"{PROVIDER_URL}:19191/api"
# PROVIDER_CONTROL = f"{PROVIDER_URL}:19192/control"
# PROVIDER_MANAGEMENT = f"{PROVIDER_URL}:19193/management"
# PROVIDER_PROTOCOL = f"{PROVIDER_URL}:19194/protocol"
# PROVIDER_PUBLIC = f"{PROVIDER_URL}:19291/public"

# CONSUMER_API = f"{CONSUMER_URL}:29191/api"
# CONSUMER_CONTROL = f"{CONSUMER_URL}:29192/control"
# CONSUMER_MANAGEMENT = f"{CONSUMER_URL}:29193/management"
# CONSUMER_PROTOCOL = f"{CONSUMER_URL}:29194/protocol"
# CONSUMER_PUBLIC = f"{CONSUMER_URL}:29291/public"


In [20]:
"""
The vars below are useful when working on local deployment
(or at least when no routings are specified for connectors)
Docker containers have their own localhost, which is not the host
machine's localhost.

In some requests there is a 'counterPartyAddress' which contains
localhost. This one will be solved as containers' internal localhost
but not the localhost of the host machine and connectors won't connect
to each other.

Hence we substitute the "localhost" with containers' names (if they
contain "localhost", otherwise urls remain unchanged).
If routings are specified correctly on PaaS, those lines aren't
required.

We leave those vars here anyway, just not to complicate 
any of the requests later in the demo. 
"""

provider_control_intern = PROVIDER_CONTROL.replace(LOCALHOST, PROVIDER_CONTAINER)
provider_public_intern = PROVIDER_PUBLIC.replace(LOCALHOST, PROVIDER_CONTAINER)
provider_protocol_internal = f"{PROVIDER_PROTOCOL}".replace(LOCALHOST, PROVIDER_CONTAINER)


### Conn Check

In [13]:
print(f"{PROVIDER_API}/check/health/")
rp = requests.get(f"{PROVIDER_API}/check/health/").json()
print(rp)

print()

print(f"{CONSUMER_API}/check/health/")
rc = requests.get(f"{CONSUMER_API}/check/health/").json()
print(rc)

print()
print("They're Alive!")

https://provider-edc-connector.apps.paas-dev.psnc.pl/api/check/health/
{'componentResults': [{'failure': None, 'component': None, 'isHealthy': True}], 'isSystemHealthy': True}

https://consumer-edc-connector.apps.paas-dev.psnc.pl/api/check/health/
{'componentResults': [{'failure': None, 'component': None, 'isHealthy': True}], 'isSystemHealthy': True}

They're Alive!


### Populating the provider

In [14]:
def register_data_plane_instance_for_provider():
    global provider_control_intern
    global provider_public_intern

    return requests.post(
            headers=default_headers,
            data=json.dumps({
                "@context": {
                    "edc": "https://w3id.org/edc/v0.0.1/ns/"
                },
                "@id": "provider-dataplane",
                "url": f"{provider_control_intern}/transfer",
                "allowedSourceTypes": [ "HttpData" ],
                "allowedDestTypes": [ "HttpProxy", "HttpData" ],
                "properties": {
                        "https://w3id.org/edc/v0.0.1/ns/publicApiUrl": f"{provider_public_intern}/"
                }
            }),
            url=f"{PROVIDER_MANAGEMENT}/v2/dataplanes"
        )

"""
better execute it once on your connector, if you don't
know what you're doing it for. I don't, but it's apparently
necessairy and overusing it may cause appearing unexpected
contract offers for exapmple.

It's not harmful at all, just has the potential to confuse the dev.
"""

register_data_plane_instance_for_provider().status_code

200

In [17]:
create_an_asset_on_the_provider_side = requests.post(
            headers=default_headers,
            data=json.dumps({
                "@context": {
                    "edc": "https://w3id.org/edc/v0.0.1/ns/"
                },
                "@id": "asset-workshop-demo-by-api",
                "properties": {
                    "name": "test of asset, just to have another one on board",
                    "contenttype": "application/json"
                    },
                "dataAddress": {
                    "name": "Test asset",
                    "baseUrl": "https://jsonplaceholder.typicode.com/users",
                    "type": "HttpData"
                    }
                }),
            url=f"{PROVIDER_MANAGEMENT}/v3/assets"
            )

# expected 200 or 409 (already exists) because the @id is fixed in the example
create_an_asset_on_the_provider_side.status_code

409

In [18]:
create_policy_on_the_provider = requests.post(
            headers=default_headers,
            data=json.dumps({
                "@context": {
                    "edc": "https://w3id.org/edc/v0.0.1/ns/",
                    "odrl": "http://www.w3.org/ns/odrl/2/"
                },
                "@id": "aPolicy",
                "policy": {
                    "@type": "set",
                    "odrl:permission": [],
                    "odrl:prohibition": [],
                    "odrl:obligation": []
                }
                }),
            url=f"{PROVIDER_MANAGEMENT}/v2/policydefinitions"
        )

# expected 200 or 409 (already exists) because the @id is fixed in the example
create_policy_on_the_provider.status_code

409

In [19]:
create_a_contract_definition_on_provider = requests.post(
            headers=default_headers,
            data=json.dumps({
                "@context": {
                    "edc": "https://w3id.org/edc/v0.0.1/ns/"
                },
                "@id": "3",
                "accessPolicyId": "aPolicy",
                "contractPolicyId": "aPolicy",
                "assetsSelector": []
                }),
            url=f"{PROVIDER_MANAGEMENT}/v2/contractdefinitions"
        )

# expected 200 or 409 (already exists) because the @id is fixed in the example
create_a_contract_definition_on_provider.status_code

409

### Working on consumer side

In [21]:
fetch_catalog_on_consumer_side = requests.post(
            headers=default_headers,
            data=json.dumps({
                "@context": {
                    "edc": "https://w3id.org/edc/v0.0.1/ns/"
                },
                "@type": "CatalogRequest",
                "counterPartyAddress": provider_protocol_internal,
                "protocol": "dataspace-protocol-http"
                }),
            url=f"{CONSUMER_MANAGEMENT}/v2/catalog/request"
        )

fetch_catalog_on_consumer_side.status_code

200

In [ ]:
# get_dataplanes_or_sth = requests.get(
#     headers=default_headers,
#     url=f"{PROVIDER_MANAGEMENT}/v2/dataplanes"
# )
# get_dataplanes_or_sth.json()

In [22]:
## a quick lookup
fetch_catalog_on_consumer_side.json()

{'@id': '91a822a9-2201-4d5c-8be9-273ec98bd115',
 '@type': 'dcat:Catalog',
 'dcat:dataset': [{'@id': 'asset-de-raul',
   '@type': 'dcat:Dataset',
   'odrl:hasPolicy': {'@id': 'Mw==:YXNzZXQtZGUtcmF1bA==:YzFkYmM5YTctMjVlNy00OGE1LWFhNjctODE1MDM3NWNmYThj',
    '@type': 'odrl:Set',
    'odrl:permission': [],
    'odrl:prohibition': [],
    'odrl:obligation': [],
    'odrl:target': 'asset-de-raul'},
   'dcat:distribution': [{'@type': 'dcat:Distribution',
     'dct:format': {'@id': 'HttpData'},
     'dcat:accessService': 'f6c9339a-af2e-499c-9b7a-6a94a518cb92'},
    {'@type': 'dcat:Distribution',
     'dct:format': {'@id': 'HttpData'},
     'dcat:accessService': 'f6c9339a-af2e-499c-9b7a-6a94a518cb92'},
    {'@type': 'dcat:Distribution',
     'dct:format': {'@id': 'HttpData'},
     'dcat:accessService': 'f6c9339a-af2e-499c-9b7a-6a94a518cb92'},
    {'@type': 'dcat:Distribution',
     'dct:format': {'@id': 'HttpData'},
     'dcat:accessService': 'f6c9339a-af2e-499c-9b7a-6a94a518cb92'},
    {'@type

In [23]:
dcat_dataset = fetch_catalog_on_consumer_side.json()["dcat:dataset"]

if isinstance(dcat_dataset, list):
    offer_id = dcat_dataset[0]["odrl:hasPolicy"]["@id"]
else:
    offer_id = dcat_dataset["odrl:hasPolicy"]["@id"]

negociate_a_contract = requests.post(
            headers=default_headers,
            data=json.dumps({
                "@context": {
                    "edc": "https://w3id.org/edc/v0.0.1/ns/",
                    "odrl": "http://www.w3.org/ns/odrl/2/"
                },
                "@type": "https://w3id.org/edc/v0.0.1/ns/ContractRequest",
                "connectorId": "provider",
                "connectorAddress": f"{provider_protocol_internal}",
                "consumerId": "consumer",
                "providerId": "provider",
                "protocol": "dataspace-protocol-http",
                "offer": {
                    "offerId": f"{offer_id}",
                    "assetId": "assetId",
                    "policy": {
                        "@id": f"{offer_id}",
                        "@type": "Set",
                        "odrl:permission": [],
                        "odrl:prohibition": [],
                        "odrl:obligation": [],
                        "odrl:target": "assetId"
                    }
                }
            }),
            url=f"{CONSUMER_MANAGEMENT}/v2/contractnegotiations"
        )
negociate_a_contract.status_code

200

In [24]:
negociate_a_contract.json()

{'@type': 'edc:IdResponse',
 '@id': 'eba846c5-f0b2-4d17-97b3-8adfdb86690e',
 'edc:createdAt': 1706881418137,
 '@context': {'dct': 'https://purl.org/dc/terms/',
  'edc': 'https://w3id.org/edc/v0.0.1/ns/',
  'dcat': 'https://www.w3.org/ns/dcat/',
  'odrl': 'http://www.w3.org/ns/odrl/2/',
  'dspace': 'https://w3id.org/dspace/v0.8/'}}

In [25]:
contract_negotiation_id = negociate_a_contract.json()["@id"]

def wait_for_negotiation_to_reach_finalized_state():
    """
    In some cases, contract negotiation may take some time to reach "finalized" state.
    This function waits for agreement to reach "FINALIZED" state (without time limit though).
    It doesn't return anything because it's job is only to wait until the connector is ready
    for further work
    """

    from time import sleep
    while True:
        getting_contract_agreement_id = requests.get(
            headers={"Content-Type": "application/json"},
            url=f"{CONSUMER_MANAGEMENT}/v2/contractnegotiations/{contract_negotiation_id}"
        )
        print(getting_contract_agreement_id.status_code)
        try:
            JJ = getting_contract_agreement_id.json()
            if "edc:state" in JJ and JJ["edc:state"] == "FINALIZED": return
        except json.JSONDecodeError:
            pass
        sleep(1.0)

contract_negotiation_id

'eba846c5-f0b2-4d17-97b3-8adfdb86690e'

In [26]:
getting_contract_agreement_id = requests.get(
        headers=default_headers,
        url=f"{CONSUMER_MANAGEMENT}/v2/contractnegotiations/{contract_negotiation_id}"
    )
getting_contract_agreement_id.json()["edc:state"]

'VERIFIED'

In [31]:
def request_provider_push_transfer(contract_agreement_id: str) -> requests.Response:
    global provider_internal
    return requests.post(
            headers={'Content-Type': 'application/json'},
            data=json.dumps({
                "@context": {
                "edc": "https://w3id.org/edc/v0.0.1/ns/"
                },
                "@type": "TransferRequestDto",
                "connectorId": "provider",
                "connectorAddress": f"{provider_protocol_internal}",
                "contractId": f"{contract_agreement_id}",
                "assetId": "assetId",
                "managedResources": False,
                "protocol": "dataspace-protocol-http",
                "dataDestination": { 
                    "type": "HttpData",
                    "baseUrl": f"http://consumer-backend:4000/api/consumer/store"
                }
            }),
            url=f"{CONSUMER_MANAGEMENT}/v2/transferprocesses"
        )

def request_consumer_pull_transfer(contract_agreement_id: str) -> requests.Response:
    global provider_internal
    return requests.post(
            headers=default_headers,
            data=json.dumps({
                "@context": {
                "@vocab": "https://w3id.org/edc/v0.0.1/ns/"
                },
                # "@type": "TransferRequestDto",
                "connectorId": "provider",
                "connectorAddress": f"{provider_protocol_internal}",
                "contractId": f"{contract_agreement_id}",
                "assetId": "assetId",
                # "managedResources": False,
                "protocol": "dataspace-protocol-http",
                "dataDestination": { 
                    "type": "HttpProxy",
                 }
            }),
            url=f"{CONSUMER_MANAGEMENT}/v2/transferprocesses"
        )


In [32]:
contract_agreement_id = getting_contract_agreement_id.json()["edc:contractAgreementId"]

# provider_push_transfer = provider_push_transfer(contract_agreement_id)
consumer_pull_transfer = request_consumer_pull_transfer(contract_agreement_id)

In [33]:
consumer_pull_transfer.json()["@id"]

'96643905-42cf-4954-bbe3-b910fcbdbabc'

In [38]:
pull_transfer_id = consumer_pull_transfer.json()["@id"]

"""
Don't be supprised if transfer remains in the "Started" state.
It may stay this way even when transfer is completed.
"""

requests.get(headers=default_headers, url=f"{CONSUMER_MANAGEMENT}/v2/transferprocesses/{pull_transfer_id}/state").json()

{'@type': 'edc:TransferState',
 'edc:state': 'STARTED',
 '@context': {'dct': 'https://purl.org/dc/terms/',
  'edc': 'https://w3id.org/edc/v0.0.1/ns/',
  'dcat': 'https://www.w3.org/ns/dcat/',
  'odrl': 'http://www.w3.org/ns/odrl/2/',
  'dspace': 'https://w3id.org/dspace/v0.8/'}}

In [35]:
consumer_pull_transfer.json()

{'@type': 'edc:IdResponse',
 '@id': '96643905-42cf-4954-bbe3-b910fcbdbabc',
 'edc:createdAt': 1706881918986,
 '@context': {'dct': 'https://purl.org/dc/terms/',
  'edc': 'https://w3id.org/edc/v0.0.1/ns/',
  'dcat': 'https://www.w3.org/ns/dcat/',
  'odrl': 'http://www.w3.org/ns/odrl/2/',
  'dspace': 'https://w3id.org/dspace/v0.8/'}}

In [36]:
transfer_process_id = consumer_pull_transfer.json()["@id"]

check_transfer_status = requests.get(f"{CONSUMER_MANAGEMENT}/v2/transferprocesses/{transfer_process_id}", headers=default_headers)

check_transfer_status.json()

{'@id': '96643905-42cf-4954-bbe3-b910fcbdbabc',
 '@type': 'edc:TransferProcess',
 'edc:correlationId': '96643905-42cf-4954-bbe3-b910fcbdbabc',
 'edc:state': 'STARTED',
 'edc:stateTimestamp': 1706881919673,
 'edc:type': 'CONSUMER',
 'edc:assetId': 'assetId',
 'edc:contractId': 'd13149c1-66dd-49cb-86a0-0d11b025626b',
 'edc:callbackAddresses': [],
 'edc:dataDestination': {'@type': 'edc:DataAddress', 'edc:type': 'HttpProxy'},
 'edc:connectorId': 'provider',
 '@context': {'dct': 'https://purl.org/dc/terms/',
  'edc': 'https://w3id.org/edc/v0.0.1/ns/',
  'dcat': 'https://www.w3.org/ns/dcat/',
  'odrl': 'http://www.w3.org/ns/odrl/2/',
  'dspace': 'https://w3id.org/dspace/v0.8/'}}